In [3]:
import sys
sys.path.append('..')
import time
from astropy.table import Table
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from os.path import expanduser
from deeplens.resnet_classifier import deeplens_classifier
from deeplens.utils.blfchallenge import classify_ground_challenge 
from sklearn import metrics
import glob
import csv
import copy
from os.path import expanduser
home = expanduser("~")

In [4]:
# specify paths

download_path=home + '/DECaLS-FindingLens/CMUDeepLens-X/sim-lens/'  # To be adjusted on your machine

# Path to export the data
export_path=home + '/DECaLS-FindingLens/CMUDeepLens-X/catalog/'   # To be adjusted on your machine


# download_path = '/Users/mac/Desktop/LBNL/challenge_data/' # (currently set for andrew)
# export_path = '/Users/mac/Desktop/LBNL/' # (currently set for andrew)

In [5]:
# specify number of images, read csv file
imgs = 1800
cat = Table.read(download_path+'GroundBasedTraining/classifications.csv')
cat = cat[0:imgs]

In [7]:
# generate catalog with specified number of images
ims = np.zeros((imgs, 3, 101, 101))
counter = 0


'''This cell take a long time to run'''

ims = np.zeros((imgs, 3, 101, 101))
counter = 0


for i, id in enumerate(cat['ID']):
    if (i%100 == 0) and (i!=0):
        counter +=1
        print str(counter)+"00 done."
    for j, b in enumerate(['R', 'I', 'G']):
        ims[i, j] = fits.getdata(download_path+'GroundBasedTraining/Public/Band'+str(j+1)+'/imageSDSS_'+b+'-'+str(id)+'.fits')
        cat['image'] = ims

cat.write(export_path+'catalogs_'+str(imgs)+'_RGB.hdf5', path='/ground', append=True)
print "Done !"



100 done.
200 done.
300 done.
400 done.
500 done.
600 done.
700 done.
800 done.
900 done.
1000 done.
1100 done.
1200 done.
1300 done.
1400 done.
1500 done.
1600 done.
1700 done.


IOError: Table /ground already exists

In [23]:
# get x and y from catalog that was just generated
d = Table.read(export_path+'catalogs_1800_RGB.hdf5', path='/ground')  # Path to be adjusted on your machine
print d['image'].shape

x = np.asarray(d['image']).reshape((-1,3,101,101))
print x.shape

y = np.asarray(d['is_lens']).reshape((-1,1))
print y.shape

(1800, 3, 101, 101)
(1800, 3, 101, 101)
(1800, 1)


In [27]:
# attempt to load 1800 model with original training set
model_18 = deeplens_classifier()
model_18.load(export_path+'catalogs_1800_RGB.hdf5', x, y)
# model_18.load(export_path+'deeplens_params1800images_3bands.npy', x, y)

loading from /Users/xhuang22/DECaLS-FindingLens/CMUDeepLens-X/catalog/catalogs_1800_RGB.hdf5...


UnpicklingError: invalid load key, 'H'.

In [ ]:
# load 1200 model
model_12 = deeplens_classifier()
model_12.load(export_path+'deeplens_params1200images_3bands.npy', x[:1200], y[:1200])

In [ ]:
# tpr,fpr,th = model.eval_ROC(x_test,y_test)

In [ ]:
# roc_auc = metrics.auc(fpr, tpr)
# plt.figure(figsize= (10,7))
# plt.title('ROC on new test set', fontsize=20)
# plt.plot(fpr,tpr)
# plt.xlabel('FPR', fontsize=15)
# plt.ylabel('TPR', fontsize=15)
# plt.xlim(0,1)
# plt.ylim(0,1.)
# plt.plot([0, 1], [0, 1], linestyle='--')
# plt.xlim([-0.01, 1.0])
# plt.ylim([0.0, 1.01])
# plt.text(1, 0.2, 'AUC: ' + str("%.4f" % roc_auc),
#         verticalalignment='bottom', horizontalalignment='right', fontsize=20)
# # plt.text(1, 0.1, 'Purity: ' + str("%.4f" % purity),
# #         verticalalignment='bottom', horizontalalignment='right', fontsize=20)
# # plt.text(1, 0, 'Completeness: ' + str("%.4f" % completeness),
# #         verticalalignment='bottom', horizontalalignment='right', fontsize=20)
# plt.grid(True)

# plt.show()